In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("VSCodeTest") \
    .master("local[*]") \
    .getOrCreate()

print("✅ Spark session running in VS Code!")
print("Spark version:", spark.version)

✅ Spark session running in VS Code!
Spark version: 4.0.0


In [17]:
from openai import OpenAI
from IPython.display import Markdown, display
import unicodedata

# Connect to your local Ollama server
client = OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="ollama"  # Dummy key required by SDK
)

def ask_local_gpt(prompt: str, model="gpt-oss:20b", system_message=None, render_markdown=True):
    messages = []
    if system_message:
        messages.append({"role": "system", "content": system_message})
    messages.append({"role": "user", "content": prompt})

    response = client.chat.completions.create(
        model=model,
        messages=messages,
    )

    raw_output = response.choices[0].message.content

    # Normalize spacing but preserve formatting
    clean_output = unicodedata.normalize("NFKC", raw_output).replace("\u202f", " ")

    if render_markdown:
        display(Markdown(clean_output))
    return clean_output


In [18]:
ask_local_gpt("What's 1 + 1? Give me the whole equation.")

The full equation is:

\[
1 + 1 = 2
\]

'The full equation is:\n\n\\[\n1 + 1 = 2\n\\]'

In [2]:
spark.stop()